In [48]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import joblib

import warnings
warnings.filterwarnings('ignore')

differences 

In [32]:
model_df = pd.read_csv('./data_frames/game_stats_differences_2020.csv')

In [33]:
model_df.head()

,goal_diff,won,game_ids,goals,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,ice_tilt
0,3,yes,2020020001,3,-7,2,0,33.4,1.0,0.0,16.0,-2,0,8,-1058.0
1,4,yes,2020020002,4,10,5,-2,16.7,1.0,1.0,1.8,0,0,2,89.0
2,1,yes,2020020003,1,2,-5,-2,-16.7,0.0,1.0,8.0,-1,-4,-18,1328.0
3,-2,no,2020020004,-2,-4,3,-4,0.0,0.0,2.0,3.6,-1,4,-1,695.0
4,-3,no,2020020005,-3,-5,-8,-4,25.0,1.0,2.0,8.8,3,5,1,405.0


In [34]:
X = model_df.drop(columns=['won', 'goal_diff','goals','game_ids', ])
y = model_df['won']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [5]:
sc = StandardScaler()

Xs_train = sc.fit_transform(X_train)
Xs_test = sc.transform(X_test)

In [6]:
logreg = LogisticRegression(random_state=42, max_iter=1000)

In [7]:
log_params = {
    'C': np.linspace(0.1,4,100)
             }

In [8]:
log_reg = RandomizedSearchCV(logreg,log_params,n_iter=100,n_jobs=-1,verbose=1)

In [9]:
log_reg.fit(Xs_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000, random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([0.1       , 0.13939394, 0.17878788, 0.21818182, 0.25757576,
       0.2969697 , 0.33636364, 0.37575758, 0.41515152, 0.45454545,
       0.49393939, 0.53333333, 0.57272727, 0.61212121, 0.65151515,
       0.69090909, 0.73030303, 0.76969697, 0.80909091, 0.84848485,
       0.8878...
       2.66060606, 2.7       , 2.73939394, 2.77878788, 2.81818182,
       2.85757576, 2.8969697 , 2.93636364, 2.97575758, 3.01515152,
       3.05454545, 3.09393939, 3.13333333, 3.17272727, 3.21212121,
       3.25151515, 3.29090909, 3.33030303, 3.36969697, 3.40909091,
       3.44848485, 3.48787879, 3.52727273, 3.56666667, 3.60606061,
       3.64545455, 3.68484848, 3.72424242, 3.76363636, 3.8030303 ,
       3.84242424, 3.88181818, 3.92121212, 3.96060606, 4.        ])},
                   verbose=1)

In [10]:
log_reg.best_params_

{'C': 0.17878787878787877}

In [11]:
log_reg.score(Xs_test,y_test), log_reg.score(Xs_train,y_train)

(0.7419354838709677, 0.6912442396313364)

In [12]:
#joblib.dump(log_reg,'./models/log_reg.pkl')

['./models/log_reg.pkl']

random forest

In [27]:
rfc = RandomForestClassifier()

In [28]:
rfc_params = {
    'n_estimators': list(range(2,51,1)),
    'max_depth': list(range(3,51,1)),
    'min_samples_split': list(range(2,51,1)),
    'min_samples_leaf': list(range(2,51,1)),
    'max_features': ['auto','sqrt','log2']
}

In [29]:
rfc_search = RandomizedSearchCV(rfc,rfc_params,n_iter=1000,n_jobs=-2,verbose=1,random_state=42)

In [35]:
rfc_search.fit(X_train,y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=1000, n_jobs=-2,
                   param_distributions={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12, 13, 14, 15, 16,
                                                      17, 18, 19, 20, 21, 22,
                                                      23, 24, 25, 26, 27, 28,
                                                      29, 30, 31, 32, ...],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [2, 3, 4, 5, 6, 7,
                                                             8, 9, 10, 11, 12,
                                                             13, 14, 15, 16, 17,
                                                             18, 19, 20, 21, 22,
                                                             23, 24, 25, 

In [36]:
rfc_search.score(X_train,y_train), rfc_search.score(X_test,y_test)

(0.7588325652841782, 0.7235023041474654)

In [ ]:
rfc_search.

In [18]:
rfc_search.best_estimator_.feature_importances_

array([0.04444509, 0.12279573, 0.01589059, 0.31355386, 0.23780591,
       0.01008572, 0.0581746 , 0.0486683 , 0.04723833, 0.0695592 ,
       0.03178267])

In [19]:
rfc_search.best_params_

{'n_estimators': 35,
 'min_samples_split': 12,
 'min_samples_leaf': 17,
 'max_features': 'log2',
 'max_depth': 3}

In [20]:
#joblib.dump(rfc_search,'./models/random_forest_classifier.pkl')

['./models/random_forest_classifier.pkl']

Minute over minute modeling:

In [23]:
two_minutes = pd.read_csv('./data_frames/200_minutes_2020.csv')

In [25]:
two_minutes_diffs = pd.read_csv('./data_frames/200_minutes_diffs_2020.csv')

In [9]:
fourtysix_minutes = pd.read_csv('./data_frames/4600_minutes_2020.csv')

In [10]:
sixty_minutes = pd.read_csv('./data_frames/6000_minutes_2020.csv')

In [22]:
sixty_minutes

,Goal,Shot,Blocked Shot,Faceoff,Takeaway,Giveaway,Hit,Missed Shot,Penalty,Stoppage,won
0,3,31,11,21,8,10,23,7,3,0,no
1,6,21,13,29,6,10,31,8,3,0,yes
2,3,31,11,21,8,10,23,7,3,0,no
3,6,21,13,29,6,10,31,8,3,0,yes
4,1,22,7,28,4,1,14,6,4,0,no
...,...,...,...,...,...,...,...,...,...,...,...
3467,2,16,12,26,1,7,23,12,3,0,NaN
3468,4,25,8,31,6,9,13,7,3,0,NaN
3469,4,43,6,30,6,10,12,9,3,0,NaN
3470,4,25,8,31,6,9,13,7,3,0,NaN


In [24]:
two_minutes

,Goal,Shot,Blocked Shot,Faceoff,Takeaway,Giveaway,Hit,Missed Shot,Penalty,won
0,0,1,1,1,0,0,1,0,0,no
1,0,2,2,2,1,0,4,0,0,yes
2,0,1,0,2,0,0,0,1,0,no
3,0,0,0,1,0,0,1,0,0,yes
4,0,2,1,0,0,2,4,0,0,no
...,...,...,...,...,...,...,...,...,...,...
1731,0,1,0,2,0,0,1,0,0,yes
1732,0,2,0,2,0,0,0,0,0,yes
1733,0,1,1,0,0,0,0,0,0,no
1734,0,2,0,1,0,0,2,1,0,yes


In [26]:
two_minutes_diffs

,Goal,Shot,Blocked Shot,Faceoff,Takeaway,Giveaway,Hit,Missed Shot,Penalty,won
0,0,1,1,1,1,0,3,0,0,yes
1,0,-1,0,-1,0,0,1,-1,0,yes
2,0,-2,-1,1,0,-2,-4,0,0,yes
3,0,0,0,-3,0,-1,-1,-1,0,no
4,0,-1,-1,-3,0,0,-1,0,-1,no
...,...,...,...,...,...,...,...,...,...,...
863,0,1,0,-1,0,0,-2,0,0,yes
864,0,0,0,0,0,0,1,0,0,yes
865,0,1,-1,2,0,0,-3,0,0,yes
866,0,-1,1,-2,0,0,0,0,0,no


differnces in stats. 

In [44]:
counter = 200
i = 0
scores_df = pd.DataFrame()
while counter <= 6000:
    df = pd.read_csv(f'./data_frames/{counter}_minutes_diffs_2020.csv')
    X = df.drop(columns='won')
    y = df['won']
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)
    rfc = RandomForestClassifier()
    rfc_params = {
    'n_estimators': list(range(2,51,1)),
    'max_depth': list(range(3,51,1)),
    'min_samples_split': list(range(2,51,1)),
    'min_samples_leaf': list(range(2,51,1)),
    'max_features': ['auto','sqrt','log2']
    }
    rfc_search = RandomizedSearchCV(rfc,rfc_params,n_iter=2000,n_jobs=-2,random_state=42)
    rfc_search.fit(X_train,y_train)
    test_score = rfc_search.score(X_test,y_test)
    train_score = rfc_search.score(X_train,y_train)
    scores = {'test_score': test_score, 'train_score': train_score, 'minutes_into_game':counter}
    temp_df = pd.DataFrame(scores, index=[i])
    scores_df = pd.concat([scores_df, temp_df ], axis=0, ignore_index=True)
    joblib.dump(rfc_search, f'./models/{counter}_rfc_search_diffs_2020.pkl')
    counter += 200
    i += 1
    
    
    

In [45]:
scores_df

,test_score,train_score,minutes_into_game
0,0.511521,0.832565,200
1,0.580645,0.800307,400
2,0.437788,0.680492,600
3,0.566820,0.737327,800
4,0.562212,0.628264,1000
5,0.571429,0.864823,1200
6,0.612903,0.649770,1400
7,0.608295,0.677419,1600
8,0.622120,0.697389,1800
9,0.585253,0.731183,2000


In [49]:
counter = 200
i = 0
log_scores_df = pd.DataFrame()
while counter <= 6000:
    df = pd.read_csv(f'./data_frames/{counter}_minutes_diffs_2020.csv')
    X = df.drop(columns='won')
    y = df['won']
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)
    sc = StandardScaler()
    Xs_train = sc.fit_transform(X_train)
    Xs_test = sc.transform(X_test)
    logreg = LogisticRegression(max_iter=1000, random_state=42)
    log_params = {'C': np.linspace(0.1,4,100), 'penalty':['l1','l2']}
    log_reg = RandomizedSearchCV(logreg,log_params,n_iter=100,n_jobs=-2, random_state=42)
    log_reg.fit(Xs_train,y_train)
    test_score = log_reg.score(Xs_test,y_test)
    train_score = log_reg.score(Xs_train,y_train)
    scores = {'test_score': test_score, 'train_score': train_score, 'minutes_into_game':counter}
    temp_df = pd.DataFrame(scores, index=[i])
    log_scores_df = pd.concat([log_scores_df, temp_df ], axis=0, ignore_index=True)
    joblib.dump(rfc_search, f'./models/{counter}_logreg_search_diffs_2020.pkl')
    counter += 200
    i += 1
    

In [50]:
log_scores_df

,test_score,train_score,minutes_into_game
0,0.520737,0.540707,200
1,0.585253,0.565284,400
2,0.516129,0.582181,600
3,0.562212,0.586790,800
4,0.576037,0.615975,1000
5,0.589862,0.652842,1200
6,0.594470,0.649770,1400
7,0.589862,0.652842,1600
8,0.589862,0.668203,1800
9,0.617512,0.697389,2000


just box scores

In [51]:
box_scores = pd.read_csv('./data_frames/2020_end_game_boxscores.csv')

In [52]:
X = box_scores.drop(columns=['won', 'team', 'home_status', 'goal_diff','goals'])
y = box_scores['won']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [53]:
logreg = LogisticRegression(max_iter=5000)
logreg.fit(X_train,y_train)

LogisticRegression(max_iter=5000)

In [54]:
logreg.score(X_test,y_test), logreg.score(X_train,y_train)

(0.6336405529953917, 0.6482334869431644)

In [55]:
joblib.dump(logreg, './models/box_score_log_reg_2020.pkl')

['./models/box_score_log_reg_2020.pkl']